# Project Guide 2 `MainActivity.java`

### **Notes and References**

Hi. I'm currently learning android development and this is my personal learning notebook. 

All the code is from Coding with Evan and you can watch the original tutorial in the below link. Have fun learning :) !

https://www.youtube.com/watch?v=sjkDbxyoNW0&ab_channel=CodingWithEvan

# New Functions & Concepts
- Request Permission
    - `ContextCompat.checkSelfPermission`
    - `ActivityCompat.requestPermissions`
- `android:layout_alignParentBottom="true"`>


# 2. `MainActivity.java`


### MA 1) Wire UI components in `.java`

- Declare the variables
- Bring them to life by initialising them in `onCreate`

In [ ]:
public class MainActivity extends AppCompatActivity {

    # Wire the components we created in UI (activity_main.xml) here in java code.
    Button button_capture, button_copy;
    TextView textview_data;      # "Hello! Take a photo or choose an image from your gallery for OCR."

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        # Initialise the wired (imported) variables.
        button_capture = findViewById(R.id.button_capture);
        button_copy = findViewById(R.id.button_copy);
        textview_data = findViewById(R.id.text_data);

### MA 2) Configure `permissions` in `AndroidManifest.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="com.example.textreaderapp">

    <uses-permission android:name="android.permission.CAMERA"/>

    ...

### MA 3) Request permission

#### `ActivityCompat`?

([Android Documentation](https://developer.android.com/reference/androidx/core/content/ContextCompat))

Helper for accessing features in `Context`.

<br/>

---

<br/>

`ContextCompat()`  
**Protected constructors**: This class should not be instantiated, but the constructor must be visible for the class to be extended.

In [ ]:
        # We are going to use the camera to capture images to perform OCR.
        # We need the permission to access camera.
           # first configure permission to camera in AndroidManifest.xml

        # # this method here handles the runtime permission.
        # If permission is not granted,
        if (ContextCompat.checkSelfPermission(MainActivity.this, Manifest.permission.CAMERA)
                != PackageManager.PERMISSION_GRANTED) {
            ActivityCompat.requestPermissions(MainActivity.this, new String[]{
                    Manifest.permission.CAMERA
            }, REQUEST_CAMERA_CODE);
        }

### MA 4) `button_capture`

- Call the Crop Image library we imported in the dependencies.
- This allows the CropImage activity;
- or the user to click anywhere in the camera or select from the gallery and crop the image..

In [ ]:
        ...

        button_capture.setOnClickListener(new View.OnClickListener() {
            @Override
            public void onClick(View v) {
                CropImage.activity().setGuidelines(CropImageView.Guidelines.ON).start(MainActivity.this);
                # Capture image result in the next method.
            }
        });

    ...

    }

### MA 5) `Crop-Image` library



#### i. Import library to `Manifest.xml`

https://github.com/ArthurHub/Android-Image-Cropper#using-activity

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android= ...

    <uses-permission android:name="android.permission.CAMERA"/>

    <application
        
        ...

        <activity
        
            ...

        </activity>

        <<activity android:name="com.theartofdev.edmodo.cropper.CropImageActivity"/>
        
    </application>

</manifest>

### MA 6) `onActivityResult`

In [ ]:
    ...

    # to capture the image result, we have to override the onActivity result method..
    @Override
    protected void onActivityResult(int requestCode, int resultCode, @Nullable Intent data) {
        super.onActivityResult(requestCode, resultCode, data);
        if (requestCode == CropImage.CROP_IMAGE_ACTIVITY_REQUEST_CODE) {
            CropImage.ActivityResult result = CropImage.getActivityResult(data);
            if (resultCode == RESULT_OK) {
                # create a uri if result is okay.
                Uri resultUri = result.getUri();
                try {
                    # create a bitmap from the uri.
                    bitmap = MediaStore.Images.Media.getBitmap(this.getContentResolver(), resultUri);
                    # Once we got the bitmap image ready, we can start OCR!

                } catch (IOException e) {
                    e.printStackTrace();
                }
            }
        }
    }

    ...

}

##### `Toast.LENGTH_SHORT` or `LONG`?

*"Answered here. Like you mentioned Toast.LENGTH_SHORT and Toast.LENGTH_LONG are not in ms but 0 or 1."*

*The actual durations are:*

https://stackoverflow.com/questions/7965135/what-is-the-duration-of-a-toast-length-long-and-length-short

In [ ]:
private static final int LONG_DELAY = 3500;    # 3.5 seconds
private static final int SHORT_DELAY = 2000;   # 2 seconds

### MA 7) `getTextFromImage`

In [ ]:
    ...

    private void getTextFromImage(Bitmap bitmap) {
        # Call text recogniser from Google Vision API
        TextRecognizer recognizer = new TextRecognizer.Builder(this).build();
        if (!recognizer.isOperational()) {
            Toast.makeText(MainActivity.this, "Error Occurred", Toast.LENGTH_SHORT).show();
        }
        else {
            Frame frame = new Frame.Builder().setBitmap(bitmap).build();
            SparseArray<TextBlock> textBlockSparseArray = recognizer.detect(frame);
            StringBuilder stringBuilder = new StringBuilder();

            for (int i=0; i<textBlockSparseArray.size(); i++){
                TextBlock textBlock = textBlockSparseArray.valueAt(i);
                stringBuilder.append(textBlock.getValue());
                stringBuilder.append("\n");
            }
            textview_data.setText(stringBuilder.toString());
            # after adding the text to the TV,
            # update button text and button_copy visibility.
            button_capture.setText("Retake");
            button_copy.setVisibility(View.VISIBLE);
        }
    }

    ...

}

### MA 8) `copyToClipBoard`

In [ ]:
    ...

    private void copyToClipBoard(String text){
        ClipboardManager clipBoard = (ClipboardManager) getSystemService(Context.CLIPBOARD_SERVICE);
        ClipData clip = ClipData.newPlainText("Copied data", text);
        clipBoard.setPrimaryClip(clip);
        Toast.makeText(MainActivity.this, "Copied to clipboard!", Toast.LENGTH_SHORT);
    }

}

### MA 9) `button_capture` & `button_copy`

In [ ]:
   ...

   @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        ...

        button_capture.setOnClickListener(new View.OnClickListener() {
            @Override
            public void onClick(View v) {
                # Call the Crop Image library we imported in the dependencies.
                # This allows the CropImage activity;
                # or the user to click anywhere in the camera or select from the gallery and crop the image..
                CropImage.activity().setGuidelines(CropImageView.Guidelines.ON).start(MainActivity.this);
                # Capture image result
            }
        });

        button_copy.setOnClickListener(new View.OnClickListener() {
            @Override
            public void onClick(View v) {
                String scanned_text = textview_data.getText().toString();
                copyToClipBoard(scanned_text);

            }
        });
    }